In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, models, layers, applications
from sklearn.model_selection import train_test_split
import wandb
from wandb.keras import WandbCallback
from matplotlib import image
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
dataset_path = "../../nature_12K/inaturalist_12K"
train_path = dataset_path + "/train"
test_path = dataset_path + "/val"
max_shape = (256, 256, 3)

data_train = ImageDataGenerator(validation_split = 0.1)#, rescale = 1. / 255)

data_test = ImageDataGenerator()#rescale = 1. / 255)

train_generator = data_train.flow_from_directory(
        train_path,
        target_size=(max_shape[0], max_shape[1]),
        class_mode='categorical',
        subset = 'training')

validation_generator = data_train.flow_from_directory(
        train_path,
        target_size=(max_shape[0], max_shape[1]),
        class_mode='categorical',
        subset = 'validation')

test_generator = data_test.flow_from_directory(
        test_path,
        target_size=(max_shape[0], max_shape[1]),
        class_mode='categorical')

Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


In [ ]:
def finetuneCNN(model_name, retrain):
    base_model = eval('applications' + model_name + '(input_shape=max_shape, include_top=False, weights=\'imagenet\')')
    base_model.trainable = True
    non_train = int((1 - retrain) * len(base_model.layers))
    for layer in base_model.layers[:non_train]:
        layer.trainable = False
    # base_model.summary()
    
    inputs = tf.keras.Input(shape=max_shape)
    #data_augmentation = tf.keras.Sequential([ 
    #layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    #layers.experimental.preprocessing.RandomRotation(0.2)])
    #x = data_augmentation(inputs)
    #x = preprocess_input(x)
    x = inputs
    x = base_model(x)
    global_average_layer = layers.GlobalAveragePooling2D()
    x = global_average_layer(x)
    prediction_layer = layers.Dense(10)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    # model.summary()
    # len(base_model.layers)
    return model

In [33]:
# Change here to change the pre-trained model

retrain = 0.2
base_model = applications.InceptionResNetV2(input_shape=max_shape, include_top=False, weights='imagenet')
base_model.trainable = True
non_train = int((1 - retrain) * len(base_model.layers))
for layer in base_model.layers[:non_train]:
    layer.trainable = False
base_model.summary()

ValueError: When setting `include_top=True` and loading `imagenet` weights, `input_shape` should be (299, 299, 3).

In [30]:
inputs = tf.keras.Input(shape=max_shape)
#data_augmentation = tf.keras.Sequential([ 
                            #layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
                            #layers.experimental.preprocessing.RandomRotation(0.2)])
#x = data_augmentation(inputs)
#x = preprocess_input(x)
x = inputs
x = base_model(x)
global_average_layer = layers.GlobalAveragePooling2D()
x = global_average_layer(x)
prediction_layer = layers.Dense(10)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)
model.summary()
len(base_model.layers)

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 256, 256, 3)]     0         
_________________________________________________________________
inception_resnet_v2 (Functio (None, 6, 6, 1536)        54336736  
_________________________________________________________________
global_average_pooling2d_8 ( (None, 1536)              0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 1536)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                15370     
Total params: 54,352,106
Trainable params: 23,028,586
Non-trainable params: 31,323,520
_________________________________________________________________


780

In [31]:
optimizer = tf.keras.optimizers.Nadam()
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

In [ ]:
model.fit(train_generator, epochs=10, validation_data=validation_generator)#, callbacks=[WandbCallback()])

Epoch 1/10
282/282 [==============================] - ETA: 0s - loss: 2.3733 - accuracy: 0.1457

In [ ]:
def train(model,
          optimizer,
          loss_fn,
          ):
    model.compile(optimizer=optimizer, loss=loss_fn, metrics = ['accuracy'])
    model.fit(train_generator, epochs=10, validation_data=validation_generator, callbacks=[WandbCallback()])

In [ ]:
sweep_config = {
    'method': 'grid',
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'
    }, 'parameters' : {
            'model' : {
                'values' : ['InceptionV3', 'InceptionResNetV2', 'ResNet50', 'Xception', 'NASNetLarge']
            }
            'retrain' : {
                'values' : [0.1, 0.15, 0.2]
            }
        }
    }

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity = '0x2e4', project = 'cs6910-a2')

In [ ]:
def run():
    default_config = {
              'model' : 'InceptionResNetV2'
                'retrain' : 0.1
           }

    run = wandb.init(project='cs6910-a2', config=default_config)
    config = wandb.config

    # initialize model
    model = finetuneCNN(model_name = config.model, retrain = config.retrain)

    # Instantiate an optimizer to train the model.
    optimizer = tf.keras.optimizers.Nadam()
    # Instantiate a loss function.
    loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    train(model,
      optimizer,
      loss_fn)

In [ ]:
wandb.agent(sweep_id, run)